In [22]:
from pymongo import MongoClient
from bson.objectid import ObjectId
from bson.son import SON
from collections import OrderedDict
from pprint import pprint
import requests
import bs4
import pandas as pd

## setup mongo connection

In [6]:
client = MongoClient("mongodb://127.0.0.1:27017/")
db = client.shops

In [7]:
serverstatus = db.command("serverStatus")
pprint(serverstatus)

{'asserts': {'msg': 0, 'regular': 0, 'rollovers': 0, 'user': 22, 'warning': 0},
 'connections': {'active': 1,
                 'available': 3272,
                 'current': 4,
                 'totalCreated': 16},
 'electionMetrics': {'averageCatchUpOps': 0.0,
                     'catchUpTakeover': {'called': 0, 'successful': 0},
                     'electionTimeout': {'called': 0, 'successful': 0},
                     'freezeTimeout': {'called': 0, 'successful': 0},
                     'numCatchUps': 0,
                     'numCatchUpsAlreadyCaughtUp': 0,
                     'numCatchUpsFailedWithError': 0,
                     'numCatchUpsFailedWithNewTerm': 0,
                     'numCatchUpsFailedWithReplSetAbortPrimaryCatchUpCmd': 0,
                     'numCatchUpsSkipped': 0,
                     'numCatchUpsSucceeded': 0,
                     'numCatchUpsTimedOut': 0,
                     'numStepDownsCausedByHigherTerm': 0,
                     'priorityTakeover': {'c

### list collections

In [10]:
db.list_collection_names()

['shops']

In [9]:
db.shops.find_one({"zip":"12169"})

{'_id': ObjectId('5e3be04cba687e77141ce276'),
 'shop': 'lidl',
 'address': 'Steglitzer Damm 95',
 'zip': '12169',
 'city': 'Berlin',
 'district': 'Steglitz',
 'closed': 1}

## scrap some data

In [80]:
name = "lidl"
zip_zone = 1
url = f"http://meinprospekt.de/filialen/{name}/{zip_zone}"

In [81]:
response = requests.get(url)

In [82]:
soup = bs4.BeautifulSoup(response.text, "html5lib")

In [105]:
shops = {}
shopid = 0

for row in soup.find_all('a', attrs={'class': 'mp-address'}):
    
    address = row.find('span', attrs={'itemprop': 'streetAddress'}).text
    zipcode = row.find('span', attrs={'itemprop': 'postalCode'}).text
    city = row.find('span', attrs={'itemprop': 'addressLocality'}).text.split("-")
    
    shops[shopid] = {
        "shop": name,
        "address": address,
        "zip": zipcode,
        "city": city[0],
        "district": city[1] if len(city) > 1 else ""
    }
    
    shopid += 1

## INSERT

#### insert_one

In [61]:
for shop in shops:
    result = db.shops.insert_one(shops[shop])
    print(f"Created {shops[shop]['address']} with ObjectID {result.inserted_id}")

Created Leipziger Str. 42 with ObjectID 5e3be04cba687e77141ce222
Created Choriner Str. 64/64a with ObjectID 5e3be04cba687e77141ce223
Created Heinrich-Heine-Str. 30 with ObjectID 5e3be04cba687e77141ce224
Created Friedenstr. 94 a with ObjectID 5e3be04cba687e77141ce225
Created Charlottenstr. 2 with ObjectID 5e3be04cba687e77141ce226
Created Luisenstr. 52 with ObjectID 5e3be04cba687e77141ce227
Created Prenzlauer Allee 44 with ObjectID 5e3be04cba687e77141ce228
Created Greifswalder Str. 33a with ObjectID 5e3be04cba687e77141ce229
Created Oranienplatz 4 with ObjectID 5e3be04cba687e77141ce22a
Created Stresemannstr. 68-78 with ObjectID 5e3be04cba687e77141ce22b
Created Kottbusser Str. 16a with ObjectID 5e3be04cba687e77141ce22c
Created Zeughofstr. 23A with ObjectID 5e3be04cba687e77141ce22d
Created Brunnenstr. 87 with ObjectID 5e3be04cba687e77141ce22e
Created Kniprodestr. 26 with ObjectID 5e3be04cba687e77141ce22f
Created Prenzlauer Allee 93 with ObjectID 5e3be04cba687e77141ce230
Created Warschauer P

#### insert_many

In [ ]:
result = db.shopsmany.insert_many([shops[shop] for shop in shops])
print(f"Created {len(result.inserted_ids)}")

can't do that again because every shop in shops received an ObjectID from mongodb on insert, that would raise a duplicate key error

In [111]:
pprint(shops[1])

{'_id': ObjectId('5e3be5b9ba687e77141ce831'),
 'address': 'Choriner Str. 64/64a',
 'city': 'Berlin',
 'district': 'Mitte',
 'shop': 'lidl',
 'zip': '10435'}


In [ ]:
result = db.shopsmany.insert_many([shops[shop] for shop in shops])
print(f"Created {len(result.inserted_ids)}")

## QUERY / filter

#### get one
by column

In [86]:
result = db.shops.find_one({"zip": "12169"})
result

{'_id': ObjectId('5e3be04cba687e77141ce276'),
 'shop': 'lidl',
 'address': 'Steglitzer Damm 95',
 'zip': '12169',
 'city': 'Berlin',
 'district': 'Steglitz'}

filter columns, if omitted, all fields will be included

In [118]:
result = db.shops.find_one({"zip": "12169"}, {"_id": 0, "address": 1, "zip": 1})
result

{'address': 'Steglitzer Damm 95', 'zip': '12169'}

#### get many

In [69]:
results = db.shops.find({"zip": "12169"})
for x in results:
    print(x)

{'_id': ObjectId('5e3be04cba687e77141ce276'), 'shop': 'lidl', 'address': 'Steglitzer Damm 95', 'zip': '12169', 'city': 'Berlin', 'district': 'Steglitz'}
{'_id': ObjectId('5e3be04cba687e77141ce277'), 'shop': 'lidl', 'address': 'Bergstr. 86', 'zip': '12169', 'city': 'Berlin', 'district': 'Steglitz'}


#### AND

In [130]:
query = {"district": "Mitte", "zip": "10435"}

results = db.shops.find(query)
list(results)[:2]

[{'_id': ObjectId('5e3be04cba687e77141ce223'),
  'shop': 'lidl',
  'address': 'Choriner Str. 64/64a',
  'zip': '10435',
  'city': 'Berlin',
  'district': 'Mitte'}]

#### OR

{ "$or": [{query}, {query}, [...] }

In [134]:
query = { "$or": [
    {"district": "Wedding"},
    {"zip": "12169"}
    ]
        }

results = db.shops.find(query)
list(results)

[{'_id': ObjectId('5e3be04cba687e77141ce23b'),
  'shop': 'lidl',
  'address': 'Reinickendorfer Str. 41',
  'zip': '13347',
  'city': 'Berlin',
  'district': 'Wedding'},
 {'_id': ObjectId('5e3be04cba687e77141ce24a'),
  'shop': 'lidl',
  'address': 'Müllerstr. 48/48a',
  'zip': '13349',
  'city': 'Berlin',
  'district': 'Wedding'},
 {'_id': ObjectId('5e3be04cba687e77141ce276'),
  'shop': 'lidl',
  'address': 'Steglitzer Damm 95',
  'zip': '12169',
  'city': 'Berlin',
  'district': 'Steglitz'}]

#### AND  as well as OR

In [144]:
query = { "city": "Berlin",
         "$or": [
    {"district": "Steglitz"},
    {"district": "Wedding"}
    ]
        }

results = db.shops.find(query)
list(results)

[{'_id': ObjectId('5e3be04cba687e77141ce23b'),
  'shop': 'lidl',
  'address': 'Reinickendorfer Str. 41',
  'zip': '13347',
  'city': 'Berlin',
  'district': 'Wedding'},
 {'_id': ObjectId('5e3be04cba687e77141ce24a'),
  'shop': 'lidl',
  'address': 'Müllerstr. 48/48a',
  'zip': '13349',
  'city': 'Berlin',
  'district': 'Wedding'},
 {'_id': ObjectId('5e3be04cba687e77141ce276'),
  'shop': 'lidl',
  'address': 'Steglitzer Damm 95',
  'zip': '12169',
  'city': 'Berlin',
  'district': 'Steglitz'},
 {'_id': ObjectId('5e3be04cba687e77141ce279'),
  'shop': 'lidl',
  'address': 'Rothenburgstr. 38a',
  'zip': '12163',
  'city': 'Berlin',
  'district': 'Steglitz'}]

#### regex

In [152]:
results = db.shops.find({"city": {"$regex": "^N"}})
list(results)[:3]

[{'_id': ObjectId('5e3be04cba687e77141ce2d6'),
  'shop': 'lidl',
  'address': 'Berliner Str. 38+40+42',
  'zip': '14641',
  'city': 'Nauen',
  'district': ''},
 {'_id': ObjectId('5e3be04cba687e77141ce2eb'),
  'shop': 'lidl',
  'address': 'Wulkower Chaussee 5',
  'zip': '16827',
  'city': 'Neuruppin',
  'district': ''},
 {'_id': ObjectId('5e3be04cba687e77141ce2ec'),
  'shop': 'lidl',
  'address': 'Neustädter Str. 53',
  'zip': '16816',
  'city': 'Neuruppin',
  'district': ''}]

#### IN
{"$in": [array]}

In [128]:
query = {"district": {"$in": ["Mitte", "Prenzlauer Berg"]}}

result = db.shops.find(query)
list(result)[:2]

[{'_id': ObjectId('5e3be04cba687e77141ce222'),
  'shop': 'lidl',
  'address': 'Leipziger Str. 42',
  'zip': '10117',
  'city': 'Berlin',
  'district': 'Mitte'},
 {'_id': ObjectId('5e3be04cba687e77141ce223'),
  'shop': 'lidl',
  'address': 'Choriner Str. 64/64a',
  'zip': '10435',
  'city': 'Berlin',
  'district': 'Mitte'}]

#### SORT results

In [155]:
query = {"district": "Mitte"}

results = db.shops.find(query).sort("zip", 1)
list(results)[:3]

[{'_id': ObjectId('5e3be04cba687e77141ce222'),
  'shop': 'lidl',
  'address': 'Leipziger Str. 42',
  'zip': '10117',
  'city': 'Berlin',
  'district': 'Mitte'},
 {'_id': ObjectId('5e3be04cba687e77141ce227'),
  'shop': 'lidl',
  'address': 'Luisenstr. 52',
  'zip': '10117',
  'city': 'Berlin',
  'district': 'Mitte'},
 {'_id': ObjectId('5e3be04cba687e77141ce224'),
  'shop': 'lidl',
  'address': 'Heinrich-Heine-Str. 30',
  'zip': '10179',
  'city': 'Berlin',
  'district': 'Mitte'}]

#### count

In [198]:
query = {"district": "Mitte"}

count = db.shops.count_documents(query)
count

4

#### limit

In [187]:
results = db.shops.find().limit(1)
list(results)

[{'_id': ObjectId('5e3be04cba687e77141ce222'),
  'shop': 'lidl',
  'address': 'Leipziger Str. 42',
  'zip': '10117',
  'city': 'Berlin',
  'district': 'Mitte',
  'closed': 0}]

#### aggregations

if sorting is of importance, we have to use bson.son.SON or collections.OrderedDict to maintain order

In [23]:
# bson.son
pipeline = [
    {"$group": {"_id": "$district", "count": {"$sum": 1}}},
    {"$sort": SON([("count", -1)])}
]

results = db.shops.aggregate(pipeline)
list(results)[:10]

[{'_id': '', 'count': 167},
 {'_id': 'Kreuzberg', 'count': 7},
 {'_id': 'Charlottenburg', 'count': 6},
 {'_id': 'Prenzlauer Berg', 'count': 6},
 {'_id': 'Friedrichshain', 'count': 6},
 {'_id': 'Marzahn', 'count': 5},
 {'_id': 'Tempelhof', 'count': 4},
 {'_id': 'Hellersdorf', 'count': 4},
 {'_id': 'Mitte', 'count': 4},
 {'_id': 'Spandau', 'count': 4}]

In [24]:
# collections.OrderedDict

pipeline = [
    {"$group": {"_id": "$district", "count": {"$sum": 1}}},
    {"$sort": OrderedDict([("count", -1)])}
]

results = db.shops.aggregate(pipeline)
list(results)[:10]

[{'_id': '', 'count': 167},
 {'_id': 'Kreuzberg', 'count': 7},
 {'_id': 'Charlottenburg', 'count': 6},
 {'_id': 'Prenzlauer Berg', 'count': 6},
 {'_id': 'Friedrichshain', 'count': 6},
 {'_id': 'Marzahn', 'count': 5},
 {'_id': 'Tempelhof', 'count': 4},
 {'_id': 'Mitte', 'count': 4},
 {'_id': 'Hellersdorf', 'count': 4},
 {'_id': 'Reinickendorf', 'count': 4}]

#### explain like i'm SQL

In [ ]:
db.command('aggregate', 'shops', pipeline=pipeline, explain=True)

## CURSOR

In [191]:
markets = db.shops.find()

print(markets.next())
print(markets.next())
print(markets.next())

markets.rewind()

print(markets.next())
print(markets.next())
print(markets.next())    

print(list(markets)[:2])

{'_id': ObjectId('5e3be04cba687e77141ce222'), 'shop': 'lidl', 'address': 'Leipziger Str. 42', 'zip': '10117', 'city': 'Berlin', 'district': 'Mitte', 'closed': 0}
{'_id': ObjectId('5e3be04cba687e77141ce223'), 'shop': 'lidl', 'address': 'Choriner Str. 64/64a', 'zip': '10435', 'city': 'Berlin', 'district': 'Mitte', 'closed': 0}
{'_id': ObjectId('5e3be04cba687e77141ce224'), 'shop': 'lidl', 'address': 'Heinrich-Heine-Str. 30', 'zip': '10179', 'city': 'Berlin', 'district': 'Mitte', 'closed': 0}
{'_id': ObjectId('5e3be04cba687e77141ce222'), 'shop': 'lidl', 'address': 'Leipziger Str. 42', 'zip': '10117', 'city': 'Berlin', 'district': 'Mitte', 'closed': 0}
{'_id': ObjectId('5e3be04cba687e77141ce223'), 'shop': 'lidl', 'address': 'Choriner Str. 64/64a', 'zip': '10435', 'city': 'Berlin', 'district': 'Mitte', 'closed': 0}
{'_id': ObjectId('5e3be04cba687e77141ce224'), 'shop': 'lidl', 'address': 'Heinrich-Heine-Str. 30', 'zip': '10179', 'city': 'Berlin', 'district': 'Mitte', 'closed': 0}
[{'_id': Obj

## UPDATE

### update one

In [174]:
query = {"zip": "12169"}

result = db.shops.find_one(query)
result

{'_id': ObjectId('5e3be04cba687e77141ce276'),
 'shop': 'lidl',
 'address': 'Steglitzer Damm 95',
 'zip': '12169',
 'city': 'Berlin',
 'district': 'Steglitz'}

In [181]:
query = {"_id": ObjectId("5e3be04cba687e77141ce276")}
values = {"$set": {"closed": 1}}

db.shops.update_one(query, values)

In [182]:
query = {"zip": "12169"}
result = db.shops.find_one(query)
result

{'_id': ObjectId('5e3be04cba687e77141ce276'),
 'shop': 'lidl',
 'address': 'Steglitzer Damm 95',
 'zip': '12169',
 'city': 'Berlin',
 'district': 'Steglitz',
 'closed': 1}

### update many

In [180]:
values = {"$set": {"closed": 0}}

results = db.shops.update_many({}, values)
results.modified_count

308

In [185]:
results = db.shops.find()
list(results)[:2]

[{'_id': ObjectId('5e3be04cba687e77141ce222'),
  'shop': 'lidl',
  'address': 'Leipziger Str. 42',
  'zip': '10117',
  'city': 'Berlin',
  'district': 'Mitte',
  'closed': 0},
 {'_id': ObjectId('5e3be04cba687e77141ce223'),
  'shop': 'lidl',
  'address': 'Choriner Str. 64/64a',
  'zip': '10435',
  'city': 'Berlin',
  'district': 'Mitte',
  'closed': 0}]

## DELETE

### delete one

In [103]:
query = {"address": "Bergstr. 86"}

result = db.shops.delete_one(query)
result.deleted_count

0

### delete many

In [104]:
result = db.shopsmany.delete_many({})
result.deleted_count

310

### drop collection

In [156]:
db.shopsmany.drop()